# ArtEmis Image Captioning - Multi-Model Training (Colab T4 GPU)

## 📋 SETUP CHECKLIST (Complete Before Running)

**Before running this notebook:**

1. **Enable GPU Runtime:**
   - Go to `Runtime` → `Change runtime type` → Select `T4 GPU`

2. **Upload Data to Google Drive:**
   - Create folder: `Google Drive/artemis-captioning/`
   - Upload these from your local machine:
     ```
     artemis-captioning/
     ├── data/
     │   ├── processed/
     │   │   ├── images/        (5000 pre-resized 128x128 images, ~57 MB)
     │   │   ├── splits/        (train.json, val.json, test.json)
     │   │   ├── captions/      (caption JSON files)
     │   │   └── vocabulary.json
     │   └── embeddings/        (optional - glove, word2vec, tfidf)
     ├── utils/                 (all Python files)
     ├── models/                (all Python files)
     └── train.py
     ```

3. **Update DRIVE_DATA_PATH** in Cell 6 if needed (default: `/content/drive/MyDrive/artemis-captioning`)

## 🚀 Training Configurations

| Config | Model | Images | Epochs | Est. Time |
|--------|-------|--------|--------|-----------|
| colab_cnn_large | CNN+LSTM (512 embed, 1024 hidden) | 15,000 | 50 | ~3-4 hours |
| colab_vit_standard | ViT (256 embed, 6 layers) | 15,000 | 50 | ~2-3 hours |
| colab_cnn_glove | CNN+LSTM + GloVe embeddings | 15,000 | 40 | ~2-3 hours |

**Total estimated time: ~8-10 hours** (run overnight or train one at a time)

In [ ]:
import torch
print(f'CUDA: {torch.cuda.is_available()}')
if torch.cuda.is_available(): print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
!pip install -q nltk gensim pillow tqdm

In [ ]:
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('wordnet', quiet=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
DRIVE_PATH = '/content/drive/MyDrive/artemis-captioning'
for f in ['data/processed/vocabulary.json', 'train.py', 'models/cnn_lstm.py']:
    print('V' if os.path.exists(f'{DRIVE_PATH}/{f}') else 'X', f)

In [ ]:
!mkdir -p /content/artemis
!cp -r "{DRIVE_PATH}/data" /content/artemis/
!cp -r "{DRIVE_PATH}/utils" /content/artemis/
!cp -r "{DRIVE_PATH}/models" /content/artemis/
!cp "{DRIVE_PATH}/train.py" /content/artemis/
print('Copied to /content/artemis/')

In [ ]:
import sys
os.chdir('/content/artemis')
sys.path.insert(0, '/content/artemis')

In [ ]:
import torch, json, time, numpy as np
from utils.text_preprocessing import TextPreprocessor
from utils.data_loader import create_data_loaders
from models.cnn_lstm import create_model as create_cnn_model
from models.vision_transformer import VisionTransformerCaptioning
from train import Trainer
print('Imports OK')

In [ ]:
DEVICE = 'cuda'
NUM_WORKERS = 2

CONFIGS = {
    'colab_cnn_large': {
        'model_type': 'cnn_lstm', 'batch_size': 32, 'num_images': 15000,
        'epochs': 50, 'learning_rate': 1e-4, 'embed_dim': 512,
        'hidden_dim': 1024, 'attention_dim': 512, 'dropout': 0.4,
    },
    'colab_vit_standard': {
        'model_type': 'vit', 'batch_size': 32, 'num_images': 15000,
        'epochs': 50, 'learning_rate': 1e-4, 'embed_dim': 256,
        'num_heads': 8, 'num_layers': 6, 'ff_dim': 1024, 'dropout': 0.1,
    },
    'colab_cnn_glove': {
        'model_type': 'cnn_lstm', 'batch_size': 32, 'num_images': 15000,
        'epochs': 40, 'learning_rate': 5e-5, 'embed_dim': 300,
        'hidden_dim': 512, 'attention_dim': 256, 'dropout': 0.3, 'use_glove': True,
    }
}

In [ ]:
text_proc = TextPreprocessor()
text_proc.load_vocabulary('data/processed/vocabulary.json')
vocab_size = text_proc.vocab_size
print(f'Vocab: {vocab_size}')

glove = None
if os.path.exists('data/embeddings/glove_embeddings.npy'):
    glove = np.load('data/embeddings/glove_embeddings.npy')
    print(f'GloVe: {glove.shape}')

In [ ]:
class LimitedLoader:
    def __init__(self, loader, max_batches):
        self.loader = loader
        self.max_batches = max_batches
        self.batch_size = loader.batch_size
        self.dataset = loader.dataset
    def __iter__(self):
        for i, b in enumerate(self.loader):
            if i >= self.max_batches: break
            yield b
    def __len__(self): return min(len(self.loader), self.max_batches)

def train_model(name, cfg):
    print(f'\n{"="*70}\nTRAINING: {name}\n{"="*70}')
    ckpt_dir, out_dir = f'checkpoints/{name}', f'outputs/{name}'
    os.makedirs(ckpt_dir, exist_ok=True)
    os.makedirs(out_dir, exist_ok=True)
    
    loaders = create_data_loaders(text_preprocessor=text_proc, batch_size=cfg['batch_size'], num_workers=NUM_WORKERS, splits=['train','val'])
    train_loader = LimitedLoader(loaders['train'], cfg['num_images']//cfg['batch_size'])
    val_loader = LimitedLoader(loaders['val'], 50)
    print(f'Batches: train={len(train_loader)}, val={len(val_loader)}')
    
    if cfg['model_type'] == 'cnn_lstm':
        emb = torch.tensor(glove, dtype=torch.float32) if cfg.get('use_glove') and glove is not None else None
        model = create_cnn_model(embedding_matrix=emb, vocab_size=vocab_size, embed_dim=cfg['embed_dim'],
            decoder_dim=cfg.get('hidden_dim',512), attention_dim=cfg.get('attention_dim',256), dropout=cfg['dropout'])
    else:
        model = VisionTransformerCaptioning(vocab_size=vocab_size, embed_dim=cfg['embed_dim'],
            num_heads=cfg['num_heads'], num_encoder_layers=cfg['num_layers'], num_decoder_layers=cfg['num_layers'],
            ff_dim=cfg['ff_dim'], max_seq_len=30, dropout=cfg['dropout'], img_size=128, patch_size=16)
    
    model = model.to(DEVICE)
    print(f'Params: {sum(p.numel() for p in model.parameters()):,}')
    
    trainer = Trainer(model=model, train_loader=train_loader, val_loader=val_loader, text_preprocessor=text_proc,
        learning_rate=cfg['learning_rate'], device=DEVICE, checkpoint_dir=ckpt_dir, output_dir=out_dir)
    
    t0 = time.time()
    history = trainer.train(num_epochs=cfg['epochs'])
    dur = (time.time()-t0)/60
    
    res = {'name': name, 'duration_min': dur, 'best_bleu': max(history['val_bleu']), 'best_loss': min(history['val_loss'])}
    with open(f'{out_dir}/results.json','w') as f: json.dump(res, f, indent=2)
    print(f'Done: {dur:.1f}min, BLEU={res["best_bleu"]:.4f}')
    return res, history

## Train Models
- Option A: Run all 3 (8-10 hrs)
- Option B: Run individual cells

In [ ]:
# OPTION A: ALL MODELS
for name, cfg in CONFIGS.items():
    try: train_model(name, cfg)
    except Exception as e: print(f'ERROR {name}: {e}')
    torch.cuda.empty_cache()

In [ ]:
# OPTION B-1: CNN Large only
train_model('colab_cnn_large', CONFIGS['colab_cnn_large'])

In [ ]:
# OPTION B-2: ViT only
train_model('colab_vit_standard', CONFIGS['colab_vit_standard'])

In [ ]:
# OPTION B-3: CNN+GloVe only
train_model('colab_cnn_glove', CONFIGS['colab_cnn_glove'])

In [ ]:
# Save to Drive
DRIVE_PATH = '/content/drive/MyDrive/artemis-captioning'
for n in CONFIGS:
    !cp -r "checkpoints/{n}" "{DRIVE_PATH}/checkpoints/" 2>/dev/null || true
    !cp -r "outputs/{n}" "{DRIVE_PATH}/outputs/" 2>/dev/null || true
print('Saved to Drive')

In [ ]:
import torch
print(f'CUDA: {torch.cuda.is_available()}')
if torch.cuda.is_available(): print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
!pip install -q nltk gensim pillow tqdm

In [ ]:
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('wordnet', quiet=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
DRIVE_PATH = '/content/drive/MyDrive/artemis-captioning'
for f in ['data/processed/vocabulary.json', 'train.py', 'models/cnn_lstm.py']:
    print('✓' if os.path.exists(f'{DRIVE_PATH}/{f}') else '✗', f)

In [ ]:
!mkdir -p /content/artemis
!cp -r "{DRIVE_PATH}/data" /content/artemis/
!cp -r "{DRIVE_PATH}/utils" /content/artemis/
!cp -r "{DRIVE_PATH}/models" /content/artemis/
!cp "{DRIVE_PATH}/train.py" /content/artemis/
print('✓ Copied')

In [ ]:
import sys
os.chdir('/content/artemis')
sys.path.insert(0, '/content/artemis')

In [ ]:
import torch, json, time, numpy as np
from utils.text_preprocessing import TextPreprocessor
from utils.data_loader import create_data_loaders
from models.cnn_lstm import create_model as create_cnn_model
from models.vision_transformer import VisionTransformerCaptioning
from train import Trainer
print('✓ Imports OK')

In [ ]:
DEVICE = 'cuda'
NUM_WORKERS = 2

CONFIGS = {
    'colab_cnn_large': {
        'model_type': 'cnn_lstm', 'batch_size': 32, 'num_images': 15000,
        'epochs': 50, 'learning_rate': 1e-4, 'embed_dim': 512,
        'hidden_dim': 1024, 'attention_dim': 512, 'dropout': 0.4,
    },
    'colab_vit_standard': {
        'model_type': 'vit', 'batch_size': 32, 'num_images': 15000,
        'epochs': 50, 'learning_rate': 1e-4, 'embed_dim': 256,
        'num_heads': 8, 'num_layers': 6, 'ff_dim': 1024, 'dropout': 0.1,
    },
    'colab_cnn_glove': {
        'model_type': 'cnn_lstm', 'batch_size': 32, 'num_images': 15000,
        'epochs': 40, 'learning_rate': 5e-5, 'embed_dim': 300,
        'hidden_dim': 512, 'attention_dim': 256, 'dropout': 0.3, 'use_glove': True,
    }
}

In [ ]:
text_proc = TextPreprocessor()
text_proc.load_vocabulary('data/processed/vocabulary.json')
vocab_size = text_proc.vocab_size
print(f'Vocab: {vocab_size}')

glove = None
if os.path.exists('data/embeddings/glove_embeddings.npy'):
    glove = np.load('data/embeddings/glove_embeddings.npy')
    print(f'GloVe: {glove.shape}')

In [ ]:
class LimitedLoader:
    def __init__(self, loader, max_batches):
        self.loader = loader
        self.max_batches = max_batches
        self.batch_size = loader.batch_size
        self.dataset = loader.dataset
    def __iter__(self):
        for i, b in enumerate(self.loader):
            if i >= self.max_batches: break
            yield b
    def __len__(self): return min(len(self.loader), self.max_batches)

def train_model(name, cfg):
    print(f'\n{"="*70}\nTRAINING: {name}\n{"="*70}')
    ckpt_dir, out_dir = f'checkpoints/{name}', f'outputs/{name}'
    os.makedirs(ckpt_dir, exist_ok=True)
    os.makedirs(out_dir, exist_ok=True)
    
    loaders = create_data_loaders(text_preprocessor=text_proc, batch_size=cfg['batch_size'], num_workers=NUM_WORKERS, splits=['train','val'])
    train_loader = LimitedLoader(loaders['train'], cfg['num_images']//cfg['batch_size'])
    val_loader = LimitedLoader(loaders['val'], 50)
    print(f'Batches: train={len(train_loader)}, val={len(val_loader)}')
    
    if cfg['model_type'] == 'cnn_lstm':
        emb = torch.tensor(glove, dtype=torch.float32) if cfg.get('use_glove') and glove is not None else None
        model = create_cnn_model(embedding_matrix=emb, vocab_size=vocab_size, embed_dim=cfg['embed_dim'],
            decoder_dim=cfg.get('hidden_dim',512), attention_dim=cfg.get('attention_dim',256), dropout=cfg['dropout'])
    else:
        model = VisionTransformerCaptioning(vocab_size=vocab_size, embed_dim=cfg['embed_dim'],
            num_heads=cfg['num_heads'], num_encoder_layers=cfg['num_layers'], num_decoder_layers=cfg['num_layers'],
            ff_dim=cfg['ff_dim'], max_seq_len=30, dropout=cfg['dropout'], img_size=128, patch_size=16)
    
    model = model.to(DEVICE)
    print(f'Params: {sum(p.numel() for p in model.parameters()):,}')
    
    trainer = Trainer(model=model, train_loader=train_loader, val_loader=val_loader, text_preprocessor=text_proc,
        learning_rate=cfg['learning_rate'], device=DEVICE, checkpoint_dir=ckpt_dir, output_dir=out_dir)
    
    t0 = time.time()
    history = trainer.train(num_epochs=cfg['epochs'])
    dur = (time.time()-t0)/60
    
    res = {'name': name, 'duration_min': dur, 'best_bleu': max(history['val_bleu']), 'best_loss': min(history['val_loss'])}
    with open(f'{out_dir}/results.json','w') as f: json.dump(res, f, indent=2)
    print(f'Done: {dur:.1f}min, BLEU={res["best_bleu"]:.4f}')
    return res, history

## Train Models
- **Option A:** Run all 3 (~8-10 hrs)
- **Option B:** Run individual cells

In [ ]:
# OPTION A: ALL MODELS
for name, cfg in CONFIGS.items():
    try: train_model(name, cfg)
    except Exception as e: print(f'ERROR {name}: {e}')
    torch.cuda.empty_cache()

In [ ]:
# OPTION B-1: CNN Large only
train_model('colab_cnn_large', CONFIGS['colab_cnn_large'])

In [ ]:
# OPTION B-2: ViT only
train_model('colab_vit_standard', CONFIGS['colab_vit_standard'])

In [ ]:
# OPTION B-3: CNN+GloVe only
train_model('colab_cnn_glove', CONFIGS['colab_cnn_glove'])

In [ ]:
# Save to Drive
DRIVE_PATH = '/content/drive/MyDrive/artemis-captioning'
for n in CONFIGS:
    if os.path.exists(f'checkpoints/{n}'): !cp -r "checkpoints/{n}" "{DRIVE_PATH}/checkpoints/"
    if os.path.exists(f'outputs/{n}'): !cp -r "outputs/{n}" "{DRIVE_PATH}/outputs/"
print('✓ Saved to Drive')

In [ ]:
# 1. Check GPU
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    raise RuntimeError("No GPU! Enable in Runtime -> Change runtime type")

In [ ]:
# 2. Install packages
!pip install -q nltk gensim pillow tqdm

In [ ]:
# 3. NLTK data
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('wordnet', quiet=True)

In [ ]:
# 4. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 5. Set paths and verify files
import os
from pathlib import Path

DRIVE_PATH = '/content/drive/MyDrive/artemis-captioning'

required = [
    'data/processed/vocabulary.json',
    'data/processed/splits/train.json',
    'train.py',
    'models/cnn_lstm.py',
    'utils/data_loader.py',
    'utils/text_preprocessing.py'
]

print("Checking files...")
for f in required:
    path = os.path.join(DRIVE_PATH, f)
    status = "✓" if os.path.exists(path) else "✗ MISSING"
    print(f"  {status} {f}")

# Count images
img_path = os.path.join(DRIVE_PATH, 'data/processed/images')
if os.path.exists(img_path):
    n = sum(1 for d in os.listdir(img_path) if os.path.isdir(os.path.join(img_path, d))
            for _ in os.listdir(os.path.join(img_path, d)) if _.endswith('.jpg'))
    print(f"  ✓ Images: ~{n} files")

In [ ]:
# 6. Copy to local for speed
!mkdir -p /content/artemis
!cp -r "{DRIVE_PATH}/data" /content/artemis/
!cp -r "{DRIVE_PATH}/utils" /content/artemis/
!cp -r "{DRIVE_PATH}/models" /content/artemis/
!cp "{DRIVE_PATH}/train.py" /content/artemis/
print("✓ Copied to /content/artemis/")

In [ ]:
# 7. Setup Python path
import sys
os.chdir('/content/artemis')
sys.path.insert(0, '/content/artemis')
print(f"Working dir: {os.getcwd()}")

In [ ]:
# 8. Imports
import torch
import json
import time
import numpy as np
from datetime import datetime

from utils.text_preprocessing import TextPreprocessor
from utils.data_loader import create_data_loaders
from models.cnn_lstm import create_model as create_cnn_model
from models.vision_transformer import VisionTransformerCaptioning
from train import Trainer

print("✓ All imports successful")

In [ ]:
# 9. Configuration
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_WORKERS = 2

CONFIGS = {
    'colab_cnn_large': {
        'model_type': 'cnn_lstm',
        'description': 'CNN+LSTM Large (512 embed, 1024 hidden)',
        'batch_size': 32,
        'num_images': 15000,
        'epochs': 50,
        'learning_rate': 1e-4,
        'embed_dim': 512,
        'hidden_dim': 1024,
        'attention_dim': 512,
        'dropout': 0.4,
    },
    'colab_vit_standard': {
        'model_type': 'vit',
        'description': 'Vision Transformer (6 layers)',
        'batch_size': 32,
        'num_images': 15000,
        'epochs': 50,
        'learning_rate': 1e-4,
        'embed_dim': 256,
        'num_heads': 8,
        'num_layers': 6,
        'ff_dim': 1024,
        'dropout': 0.1,
    },
    'colab_cnn_glove': {
        'model_type': 'cnn_lstm',
        'description': 'CNN+LSTM with GloVe embeddings',
        'batch_size': 32,
        'num_images': 15000,
        'epochs': 40,
        'learning_rate': 5e-5,
        'embed_dim': 300,
        'hidden_dim': 512,
        'attention_dim': 256,
        'dropout': 0.3,
        'use_glove': True,
    }
}

print("Configurations:")
for name, cfg in CONFIGS.items():
    print(f"  {name}: {cfg['description']}")

In [ ]:
# 10. Load vocabulary
text_proc = TextPreprocessor()
text_proc.load_vocabulary('data/processed/vocabulary.json')
vocab_size = text_proc.vocab_size
print(f"Vocabulary size: {vocab_size}")

# Load GloVe if available
glove_embeddings = None
if os.path.exists('data/embeddings/glove_embeddings.npy'):
    glove_embeddings = np.load('data/embeddings/glove_embeddings.npy')
    print(f"✓ GloVe: {glove_embeddings.shape}")

In [ ]:
# 11. Helper classes and functions

class LimitedLoader:
    """Wrapper to limit batches per epoch."""
    def __init__(self, loader, max_batches):
        self.loader = loader
        self.max_batches = max_batches
        self.batch_size = loader.batch_size
        self.dataset = loader.dataset

    def __iter__(self):
        for i, batch in enumerate(self.loader):
            if i >= self.max_batches:
                break
            yield batch

    def __len__(self):
        return min(len(self.loader), self.max_batches)


def train_model(config_name, config):
    """Train a single model."""
    print("\n" + "=" * 70)
    print(f"TRAINING: {config_name}")
    print(f"{config['description']}")
    print("=" * 70)

    checkpoint_dir = f'checkpoints/{config_name}'
    output_dir = f'outputs/{config_name}'
    os.makedirs(checkpoint_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)

    # Save config
    with open(f'{output_dir}/config.json', 'w') as f:
        json.dump(config, f, indent=2)

    # Data loaders (using correct API)
    print("\nCreating data loaders...")
    loaders = create_data_loaders(
        text_preprocessor=text_proc,
        batch_size=config['batch_size'],
        num_workers=NUM_WORKERS,
        splits=['train', 'val']
    )

    max_train = config['num_images'] // config['batch_size']
    max_val = max(30, max_train // 5)

    train_loader = LimitedLoader(loaders['train'], max_train)
    val_loader = LimitedLoader(loaders['val'], max_val)
    print(f"  Train: {len(train_loader)} batches, Val: {len(val_loader)} batches")

    # Create model
    print("\nCreating model...")
    if config['model_type'] == 'cnn_lstm':
        embeddings = None
        if config.get('use_glove') and glove_embeddings is not None:
            embeddings = torch.tensor(glove_embeddings, dtype=torch.float32)
            print("  Using GloVe embeddings")

        model = create_cnn_model(
            embedding_matrix=embeddings,
            vocab_size=vocab_size,
            embed_dim=config['embed_dim'],
            decoder_dim=config.get('hidden_dim', 512),
            attention_dim=config.get('attention_dim', 256),
            dropout=config['dropout'],
            pretrained_cnn=True
        )
    else:
        model = VisionTransformerCaptioning(
            vocab_size=vocab_size,
            embed_dim=config['embed_dim'],
            num_heads=config['num_heads'],
            num_encoder_layers=config['num_layers'],
            num_decoder_layers=config['num_layers'],
            ff_dim=config['ff_dim'],
            max_seq_len=30,
            dropout=config['dropout'],
            img_size=128,
            patch_size=16
        )

    model = model.to(DEVICE)
    params = sum(p.numel() for p in model.parameters())
    print(f"  Parameters: {params:,}")

    # Trainer (learning_rate in __init__)
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        text_preprocessor=text_proc,
        learning_rate=config['learning_rate'],
        device=DEVICE,
        checkpoint_dir=checkpoint_dir,
        output_dir=output_dir
    )

    # Train (num_epochs in train())
    print(f"\nTraining for {config['epochs']} epochs...")
    start = time.time()
    history = trainer.train(num_epochs=config['epochs'])
    duration = time.time() - start

    # Results
    results = {
        'config_name': config_name,
        'description': config['description'],
        'parameters': params,
        'epochs': config['epochs'],
        'duration_minutes': duration / 60,
        'final_train_loss': history['train_loss'][-1],
        'final_val_loss': history['val_loss'][-1],
        'best_val_loss': min(history['val_loss']),
        'best_bleu': max(history['val_bleu']),
    }

    with open(f'{output_dir}/results.json', 'w') as f:
        json.dump(results, f, indent=2)

    print(f"\n{'=' * 70}")
    print(f"DONE: {config_name}")
    print(f"Duration: {duration/60:.1f} min, Best BLEU: {results['best_bleu']:.4f}")
    print(f"{'=' * 70}")

    return results, history

## Train Models

**Option A:** Run all 3 sequentially (~8-10 hours)

**Option B:** Run individual cells below

In [ ]:
# OPTION A: Train ALL models
all_results = {}
all_history = {}

for name, cfg in CONFIGS.items():
    try:
        results, history = train_model(name, cfg)
        all_results[name] = results
        all_history[name] = history
    except Exception as e:
        print(f"❌ {name} failed: {e}")
        import traceback
        traceback.print_exc()

    # Clear GPU
    torch.cuda.empty_cache()

print("\n" + "=" * 70)
print("ALL TRAINING COMPLETE!")

In [ ]:
# OPTION B-1: Train CNN+LSTM Large only (~3-4 hrs)
name = 'colab_cnn_large'
results_1, history_1 = train_model(name, CONFIGS[name])

In [ ]:
# OPTION B-2: Train ViT only (~2-3 hrs)
name = 'colab_vit_standard'
results_2, history_2 = train_model(name, CONFIGS[name])

In [ ]:
# OPTION B-3: Train CNN+GloVe only (~2-3 hrs)
name = 'colab_cnn_glove'
results_3, history_3 = train_model(name, CONFIGS[name])

## Save Results to Google Drive

In [ ]:
# Copy results back to Drive
DRIVE_PATH = '/content/drive/MyDrive/artemis-captioning'

!mkdir -p "{DRIVE_PATH}/checkpoints"
!mkdir -p "{DRIVE_PATH}/outputs"

for name in CONFIGS.keys():
    if os.path.exists(f'checkpoints/{name}'):
        !cp -r "checkpoints/{name}" "{DRIVE_PATH}/checkpoints/"
        print(f"✓ Saved checkpoints/{name}")
    if os.path.exists(f'outputs/{name}'):
        !cp -r "outputs/{name}" "{DRIVE_PATH}/outputs/"
        print(f"✓ Saved outputs/{name}")

print(f"\n✓ All saved to {DRIVE_PATH}")

## 1. Setup Environment

In [ ]:
# Check GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    raise RuntimeError("No GPU available! Please enable GPU in Runtime -> Change runtime type")

In [ ]:
# Install required packages
!pip install -q nltk gensim pillow tqdm

In [ ]:
# Download NLTK data
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('wordnet', quiet=True)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set up paths - MODIFY THIS to match your Drive folder
import os
from pathlib import Path

DRIVE_DATA_PATH = '/content/drive/MyDrive/artemis-captioning'

# Check if data exists
required_files = [
    'data/processed/vocabulary.json',
    'data/processed/splits/train.json',
    'train.py',
    'models/cnn_lstm.py',
    'utils/data_loader.py'
]

print("Checking required files...")
all_found = True
for f in required_files:
    path = os.path.join(DRIVE_DATA_PATH, f)
    if os.path.exists(path):
        print(f"  ✓ {f}")
    else:
        print(f"  ✗ {f} - NOT FOUND")
        all_found = False

# Check for preprocessed images
preprocessed_path = os.path.join(DRIVE_DATA_PATH, 'data/processed/images')
if os.path.exists(preprocessed_path) and os.listdir(preprocessed_path):
    num_imgs = sum(1 for _ in Path(preprocessed_path).rglob('*.jpg'))
    print(f"  ✓ Preprocessed images: {num_imgs} files")
else:
    print(f"  ⚠ No preprocessed images found - will use raw wikiart")

if not all_found:
    raise FileNotFoundError("Missing required files! See above.")

In [ ]:
# Copy data to local storage for faster access
!mkdir -p /content/artemis
!cp -r "{DRIVE_DATA_PATH}/data" /content/artemis/
!cp -r "{DRIVE_DATA_PATH}/utils" /content/artemis/
!cp -r "{DRIVE_DATA_PATH}/models" /content/artemis/
!cp "{DRIVE_DATA_PATH}/train.py" /content/artemis/

print("✓ Data copied to /content/artemis/")

In [ ]:
# Setup Python path
import sys
os.chdir('/content/artemis')
sys.path.insert(0, '/content/artemis')
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Import modules
import torch
import torch.nn as nn
import torch.optim as optim
import json
import time
from datetime import datetime
import numpy as np

from utils.data_loader import create_dataloaders
from utils.evaluation import BLEUScore
from train import Trainer

print("✓ All modules imported")

## 2. Configuration

In [ ]:
# Global settings
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_WORKERS = 2

# Three model configurations for Colab
COLAB_CONFIGS = {
    'colab_cnn_large': {
        'model_type': 'cnn_lstm',
        'description': 'CNN+LSTM Large Capacity',
        'batch_size': 32,
        'num_images': 15000,
        'epochs': 50,
        'learning_rate': 1e-4,
        'embed_dim': 512,
        'hidden_dim': 1024,
        'attention_dim': 512,
        'dropout': 0.4,
        'encoder_lr_factor': 0.1,
    },
    'colab_vit_standard': {
        'model_type': 'vit',
        'description': 'Vision Transformer Standard',
        'batch_size': 32,
        'num_images': 15000,
        'epochs': 50,
        'learning_rate': 1e-4,
        'embed_dim': 256,
        'num_heads': 8,
        'num_layers': 6,
        'ff_dim': 1024,
        'dropout': 0.1,
    },
    'colab_cnn_glove': {
        'model_type': 'cnn_lstm',
        'description': 'CNN+LSTM with GloVe Embeddings',
        'batch_size': 32,
        'num_images': 15000,
        'epochs': 40,  # Slightly fewer since pretrained embeddings converge faster
        'learning_rate': 5e-5,  # Lower LR for pretrained embeddings
        'embed_dim': 300,  # GloVe dimension
        'hidden_dim': 512,
        'attention_dim': 256,
        'dropout': 0.3,
        'encoder_lr_factor': 0.1,
        'use_glove': True,
    }
}

print("Training Configurations:")
print("=" * 70)
for name, cfg in COLAB_CONFIGS.items():
    print(f"\n{name}: {cfg['description']}")
    print(f"  Model: {cfg['model_type']}")
    print(f"  Images: ~{cfg['num_images']}, Epochs: {cfg['epochs']}")
    print(f"  Batch size: {cfg['batch_size']}, LR: {cfg['learning_rate']}")

In [ ]:
# Load vocabulary
with open('data/processed/vocabulary.json', 'r', encoding='utf-8') as f:
    vocab_data = json.load(f)

vocab_size = vocab_data['vocab_size']
word_to_idx = vocab_data['word2idx']
idx_to_word = {int(k): v for k, v in vocab_data['idx2word'].items()}

print(f"Vocabulary size: {vocab_size}")

In [ ]:
# Load GloVe embeddings if available
glove_embeddings = None
glove_path = 'data/embeddings/glove_embeddings.npy'
if os.path.exists(glove_path):
    glove_embeddings = np.load(glove_path)
    print(f"✓ GloVe embeddings loaded: {glove_embeddings.shape}")
else:
    print("⚠ GloVe embeddings not found - will use random initialization")

## 3. Helper Functions

In [ ]:
def create_cnn_lstm_model(config):
    """Create CNN+LSTM model."""
    from models.cnn_lstm import ImageCaptioningModel
    
    model = ImageCaptioningModel(
        embed_dim=config['embed_dim'],
        attention_dim=config['attention_dim'],
        decoder_dim=config['hidden_dim'],
        vocab_size=vocab_size,
        encoder_dim=2048,
        dropout=config['dropout'],
        pretrained_encoder=True
    )
    
    # Load GloVe embeddings if specified
    if config.get('use_glove') and glove_embeddings is not None:
        model.decoder.embedding.weight.data.copy_(
            torch.tensor(glove_embeddings, dtype=torch.float32)
        )
        print("  ✓ GloVe embeddings loaded into model")
    
    return model


def create_vit_model(config):
    """Create Vision Transformer model."""
    from models.vision_transformer import VisionTransformerCaptioning
    
    model = VisionTransformerCaptioning(
        vocab_size=vocab_size,
        embed_dim=config['embed_dim'],
        num_heads=config['num_heads'],
        num_encoder_layers=config['num_layers'],
        num_decoder_layers=config['num_layers'],
        ff_dim=config['ff_dim'],
        max_seq_len=30,
        dropout=config['dropout'],
        img_size=128,
        patch_size=16
    )
    return model


class LimitedLoader:
    """Wrapper to limit batches per epoch."""
    def __init__(self, loader, max_batches):
        self.loader = loader
        self.max_batches = max_batches
    
    def __iter__(self):
        for i, batch in enumerate(self.loader):
            if i >= self.max_batches:
                break
            yield batch
    
    def __len__(self):
        return min(len(self.loader), self.max_batches)


def train_model(config_name, config):
    """Train a single model configuration."""
    print("\n" + "=" * 70)
    print(f"TRAINING: {config_name}")
    print(f"Description: {config['description']}")
    print("=" * 70)
    
    # Create directories
    checkpoint_dir = f'checkpoints/{config_name}'
    output_dir = f'outputs/{config_name}'
    os.makedirs(checkpoint_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)
    
    # Save config
    with open(f'{output_dir}/config.json', 'w') as f:
        json.dump(config, f, indent=2)
    
    # Create data loaders
    print("\nCreating data loaders...")
    train_loader, val_loader, _ = create_dataloaders(
        images_dir='data/processed/images',
        captions_dir='data/processed/captions',
        splits_dir='data/processed/splits',
        vocab_file='data/processed/vocabulary.json',
        batch_size=config['batch_size'],
        num_workers=NUM_WORKERS
    )
    
    # Limit batches
    max_train_batches = config['num_images'] // config['batch_size']
    max_val_batches = max(20, max_train_batches // 5)
    
    train_loader = LimitedLoader(train_loader, max_train_batches)
    val_loader = LimitedLoader(val_loader, max_val_batches)
    
    print(f"  Train batches: {len(train_loader)} (~{len(train_loader) * config['batch_size']} images)")
    print(f"  Val batches: {len(val_loader)}")
    
    # Create model
    print("\nCreating model...")
    if config['model_type'] == 'cnn_lstm':
        model = create_cnn_lstm_model(config)
    else:
        model = create_vit_model(config)
    
    model = model.to(DEVICE)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"  Parameters: {total_params:,}")
    
    # Create optimizer
    if config['model_type'] == 'cnn_lstm':
        encoder_params = list(model.encoder.parameters())
        decoder_params = list(model.decoder.parameters()) + list(model.attention.parameters())
        optimizer = torch.optim.Adam([
            {'params': encoder_params, 'lr': config['learning_rate'] * config['encoder_lr_factor']},
            {'params': decoder_params, 'lr': config['learning_rate']}
        ], weight_decay=1e-5)
    else:
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=config['learning_rate'],
            weight_decay=1e-5
        )
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=3, verbose=True
    )
    
    # Create trainer
    bleu_scorer = BLEUScore(idx_to_word, word_to_idx)
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        optimizer=optimizer,
        scheduler=scheduler,
        evaluator=bleu_scorer,
        device=DEVICE,
        checkpoint_dir=checkpoint_dir,
        grad_clip=5.0
    )
    
    # Train
    print(f"\nStarting training for {config['epochs']} epochs...")
    start_time = time.time()
    history = trainer.train(num_epochs=config['epochs'])
    duration = time.time() - start_time
    
    # Save results
    results = {
        'config_name': config_name,
        'description': config['description'],
        'model_type': config['model_type'],
        'num_images': config['num_images'],
        'epochs': config['epochs'],
        'parameters': total_params,
        'final_train_loss': history[-1]['train_loss'],
        'final_val_loss': history[-1]['val_loss'],
        'best_val_loss': min(h['val_loss'] for h in history),
        'best_bleu': max(h.get('bleu', 0) for h in history),
        'duration_minutes': duration / 60,
        'history': history
    }
    
    with open(f'{output_dir}/results.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"\n{'=' * 70}")
    print(f"COMPLETED: {config_name}")
    print(f"Duration: {duration/60:.1f} minutes")
    print(f"Best Val Loss: {results['best_val_loss']:.4f}")
    print(f"Best BLEU: {results['best_bleu']:.4f}")
    print(f"{'=' * 70}")
    
    return results

## 4. Train Models

**Option A:** Run next cell to train ALL 3 models sequentially (~8-10 hours total)

**Option B:** Use the individual training cells below to train one at a time

In [ ]:
# Train all three configurations
all_results = {}

for config_name, config in COLAB_CONFIGS.items():
    try:
        results = train_model(config_name, config)
        all_results[config_name] = results
    except Exception as e:
        print(f"\n❌ Error training {config_name}: {e}")
        import traceback
        traceback.print_exc()
        all_results[config_name] = {'error': str(e)}
    
    # Clear GPU memory between models
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("\n" + "=" * 70)
print("ALL TRAINING COMPLETE!")
print("=" * 70)

### Option B: Train Individual Models (run only ONE of these cells)

In [ ]:
# Train Model 1: CNN+LSTM Large (~3-4 hours)
config_name = 'colab_cnn_large'
results_1 = train_model(config_name, COLAB_CONFIGS[config_name])
print(f"\n✅ Model 1 complete! Best BLEU: {results_1['best_bleu']:.4f}")

In [ ]:
# Train Model 2: ViT Standard (~2-3 hours)
config_name = 'colab_vit_standard'
results_2 = train_model(config_name, COLAB_CONFIGS[config_name])
print(f"\n✅ Model 2 complete! Best BLEU: {results_2['best_bleu']:.4f}")

In [ ]:
# Train Model 3: CNN+LSTM with GloVe (~2-3 hours)
config_name = 'colab_cnn_glove'
results_3 = train_model(config_name, COLAB_CONFIGS[config_name])
print(f"\n✅ Model 3 complete! Best BLEU: {results_3['best_bleu']:.4f}")

## 5. Results Summary

In [ ]:
# Print summary
print("\n" + "=" * 70)
print("TRAINING RESULTS SUMMARY")
print("=" * 70)

for name, results in all_results.items():
    if 'error' in results:
        print(f"\n{name}: FAILED - {results['error']}")
    else:
        print(f"\n{name}: {results['description']}")
        print(f"  Parameters: {results['parameters']:,}")
        print(f"  Duration: {results['duration_minutes']:.1f} minutes")
        print(f"  Final Val Loss: {results['final_val_loss']:.4f}")
        print(f"  Best BLEU: {results['best_bleu']:.4f}")

# Save combined results
with open('outputs/colab_all_results.json', 'w') as f:
    # Convert history to serializable format
    save_results = {}
    for name, res in all_results.items():
        if 'error' not in res:
            save_results[name] = {k: v for k, v in res.items() if k != 'history'}
        else:
            save_results[name] = res
    json.dump(save_results, f, indent=2)

print("\n✓ Results saved to outputs/colab_all_results.json")

In [ ]:
# Plot training curves
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = ['blue', 'green', 'orange']
for i, (name, results) in enumerate(all_results.items()):
    if 'error' in results:
        continue
    
    history = results['history']
    epochs = range(1, len(history) + 1)
    val_losses = [h['val_loss'] for h in history]
    bleu_scores = [h.get('bleu', 0) for h in history]
    
    axes[0].plot(epochs, val_losses, color=colors[i], label=name)
    axes[1].plot(epochs, bleu_scores, color=colors[i], label=name)

axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Validation Loss')
axes[0].set_title('Validation Loss Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('BLEU Score')
axes[1].set_title('BLEU Score Comparison')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/colab_training_comparison.png', dpi=150)
plt.show()

## 6. Save to Google Drive

In [ ]:
# Copy results to Google Drive
!mkdir -p "{DRIVE_DATA_PATH}/checkpoints"
!mkdir -p "{DRIVE_DATA_PATH}/outputs"

# Copy all checkpoint and output folders
for name in COLAB_CONFIGS.keys():
    if os.path.exists(f'checkpoints/{name}'):
        !cp -r "checkpoints/{name}" "{DRIVE_DATA_PATH}/checkpoints/"
    if os.path.exists(f'outputs/{name}'):
        !cp -r "outputs/{name}" "{DRIVE_DATA_PATH}/outputs/"

# Copy combined results
!cp outputs/colab_all_results.json "{DRIVE_DATA_PATH}/outputs/"
!cp outputs/colab_training_comparison.png "{DRIVE_DATA_PATH}/outputs/"

print("\n✓ All results saved to Google Drive!")
print(f"Location: {DRIVE_DATA_PATH}")